In [1]:
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# 训练集
train_dataset = datasets.MNIST(root='./',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
# 测试集
test_dataset = datasets.MNIST(root='./',
                               train=False,
                               transform=transforms.ToTensor(),
                               download=True)

In [3]:
# 批次大小
Batch_size = 64

# 装载训练集
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=Batch_size,
                          shuffle=True)

# 装载训练集
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=Batch_size,
                          shuffle=True)

In [4]:
for i, data in enumerate(train_loader):
    inputs, labels = data
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [5]:
inputs.size()

torch.Size([64, 1, 28, 28])

In [6]:
# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(784, 500), nn.Dropout(p=0),
                                    nn.Tanh())
        self.layer2 = nn.Sequential(nn.Linear(500, 300), nn.Dropout(p=0), 
                                    nn.Tanh())
        self.layer3 = nn.Sequential(nn.Linear(300, 10), nn.Softmax(dim=1))

    def forward(self, x):
        # ([64, 1, 28, 28]) - > (64 , 784)
        x = x.view(x.size()[0], -1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [7]:
LR = 0.5
model = Net()
CrossEntropy_loss = nn.CrossEntropyLoss()
# 定义优化器，设置L2化
optimizer = optim.SGD(model.parameters(), LR, weight_decay=0.0001)

In [8]:
def train():
    # 训练状态
    model.train()
    for i, data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果(64, 10)
        out = model(inputs)
        # 交叉熵代价函数out(batch,C),labels(batch)
        loss = CrossEntropy_loss(out, labels)
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()


# 用于模型测试
def test():
    # 模型的测试状态（此状态下，模型的dropout层不起作用，所有神经元都要工作）
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果(64, 10)
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item()/len(test_dataset)))

    # 同上，将训练集合测试集的准确率都表示出来
    correct = 0
    for i, data in enumerate(train_loader):
        # 获得一个批次的数据和标签
        inputs, labels = data
        # 获得模型预测结果(64, 10)
        out = model(inputs)
        # 获得最大值，以及最大值所在的位置
        _, predicted = torch.max(out, 1)
        # 预测正确的数量
        correct += (predicted == labels).sum()
    print("Train acc:{0}".format(correct.item()/len(train_dataset)))

#### 小结：
> 手动打出这个训练流程你会发现相对于keras这种高度集成的API，torch需要从头到尾的用代码去实现神经网络的训练过程，所以其中有很多细节可以修改，当然之后可以尝试学习autoML，可以更快的调参

> 可以看出pytorch和tensorflow各有各的好处，比如tensorboard的使用，先别管先把手上的一些小任务小领域先完成

In [9]:
for epoch in range(10):
    print('epoch:', epoch)
    train()
    test()

epoch: 0
Test acc:0.9199
Train acc:0.9150666666666667
epoch: 1
Test acc:0.9274
Train acc:0.9288666666666666
epoch: 2


KeyboardInterrupt: 